In [1]:
# importing all the necessary libraries

import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
# reading the dataset 

df= pd.read_csv("train.csv")

# reading the first 5 rows of the datset
display(df.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# dropping the unnecessary columns from the dataset

df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

# Checking the df after dropping the columns
display(df)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [6]:
# Step 1  (Train and Test Split)

X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=["Survived"]),df["Survived"],test_size=0.2,random_state=42)

In [13]:
# printing first 5 rows of X_train,X_test,y_train,y_test

def display_train_test_data():
    print(X_train.head())
    print ("---------------------------------------------------------------------------")
    print(X_test.head())
    print ("---------------------------------------------------------------------------")
    print(y_train.head())
    print ("---------------------------------------------------------------------------")
    print(y_test.head())
    
    return display_train_test_data

# calling the function display_train_test_data
display_train_test_data()

     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
331       1    male  45.5      0      0  28.5000        S
733       2    male  23.0      0      0  13.0000        S
382       3    male  32.0      0      0   7.9250        S
704       3    male  26.0      1      0   7.8542        S
813       3  female   6.0      4      2  31.2750        S
---------------------------------------------------------------------------
     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
709       3    male   NaN      1      1  15.2458        C
439       2    male  31.0      0      0  10.5000        S
840       3    male  20.0      0      0   7.9250        S
720       2  female   6.0      0      1  33.0000        S
39        3  female  14.0      1      0  11.2417        C
---------------------------------------------------------------------------
331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64
---------------------------------------------------------------------------
70

<function __main__.display_train_test_data()>

In [15]:
# checking for the missing values in the dataset

df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [19]:
# Applying imputation to fill the missing values

#creating two objects of simpleimputer class
si_age=SimpleImputer()
si_embarked=SimpleImputer(strategy='most_frequent')

X_train_age = si_age.fit_transform(X_train[["Age"]])
X_train_embarked=si_embarked.fit_transform(X_train[["Embarked"]])

X_test_age=si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

In [21]:
# One hot encoding on embarked and sex columns

#creating two seperate objects of OneHotEncoder 

ohe_sex=OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked=OneHotEncoder(sparse=False,handle_unknown='ignore')

X_train_sex= ohe_sex.fit_transform(X_train[["Sex"]])
X_train_embarked=ohe_embarked.fit_transform(X_train_embarked)

X_test_sex=ohe_sex.transform(X_test[["Sex"]])
X_test_embarked=ohe_embarked.transform(X_test_embarked)

In [27]:
# At this point of time after tranformation of previous two things like imputation and one hot encoding we have 
# 3 different arrays as of now i.e "X_train_age","X_train_embarked","X_train_sex" 

# Now we need to drop the columns "Sex","Age" and "Embarked" from the train and test data and concatinate these new three arrays 

# creating X_train remaining dataframe, X_test_remaining dataframe

X_train_rem=X_train.drop(columns=["Sex","Age","Embarked"])
X_test_rem=X_test.drop(columns=["Sex","Age","Embarked"])

# now concatinating all the columns to form a new df (X_train_rem,X_train_age,X_train_sex,X_train_embarked) same for (X_test_rem,X_test_age,X_test_sex,X_test_embarked)

X_train_transformed=np.concatenate((X_train_rem,X_train_age,X_train_sex,X_train_embarked),axis=1)
X_test_transformed=np.concatenate((X_test_rem,X_test_age,X_test_sex,X_test_embarked),axis=1)

In [31]:
# checking the shape of the transfored df's

def display_transdf_shape():
    print(X_train_transformed.shape)
    print("-----------------------------")
    print(X_test_transformed.shape)

# calling the function
display_transdf_shape()

(712, 10)
-----------------------------
(179, 10)


In [33]:
# creating a decision tree classification model

clf=DecisionTreeClassifier()

#fitting the training df

clf.fit(X_train_transformed,y_train)

# predicting the y_pred using X_test_transformed

y_pred=clf.predict(X_test_transformed)
print(y_pred)

[0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1
 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 1
 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1
 0 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1
 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1]


In [34]:
# calculating accuracy score

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7821229050279329

In [36]:
# creating pickle file for this model in onder to use it to deploy on some website

import pickle

# we will dump three files 1) ohe_sx 2) ohe_embarked 3) clf

pickle.dump(ohe_sex,open("models/ohe_sex.pkl",'wb'))
pickle.dump(ohe_embarked,open("models/ohe_embarked.pkl",'wb'))
pickle.dump(clf,open("models/clf.pkl",'wb'))